# Data Preparation

### The Target - ALPACA IYR TICKER DATA CORRELATION TO CASE-SCHILLER HOUSE PRICE INDEX

In [1]:
# Initial Imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
%matplotlib inline
import json
import datetime as dt

In [2]:
# Load .env enviroment variables
load_dotenv()

True

In [3]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

#Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [4]:
# Format current date as ISO format
start_date = pd.Timestamp("2010-05-05", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-05-05", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["IYR"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for IYR
df = api.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df

# Preview DataFrame
df =pd.DataFrame(df,index=df.index)
df.index = pd.to_datetime(df.index)
df

IYR                                 
                            open   high      low  close    volume
time                                                             
2016-05-16 00:00:00-04:00  78.56  79.52  78.4500  79.30   5131924
2016-05-17 00:00:00-04:00  78.97  79.03  77.6000  78.03   8676173
2016-05-18 00:00:00-04:00  77.65  77.85  76.0707  76.97  11817850
2016-05-19 00:00:00-04:00  76.47  76.56  75.7300  76.15   8846700
2016-05-20 00:00:00-04:00  76.10  76.84  76.1000  76.75   6980490
...                          ...    ...      ...    ...       ...
2020-04-29 00:00:00-04:00  77.35  77.85  75.7589  76.92  13755872
2020-04-30 00:00:00-04:00  75.68  76.30  74.7900  76.04  16334704
2020-05-01 00:00:00-04:00  74.41  74.42  72.7800  73.46  10784604
2020-05-04 00:00:00-04:00  72.66  73.34  71.5900  73.22   9565598
2020-05-05 00:00:00-04:00  73.87  74.76  73.5500  73.65  10374452

[1000 rows x 5 columns]

In [5]:
IYR_df = df['IYR']['close']
IYR_df =pd.DataFrame(IYR_df,index=IYR_df.index)
IYR_df.index = pd.to_datetime(IYR_df.index)
IYR_df = IYR_df.resample('Q').mean()
IYR_df.index=IYR_df.index.date
IYR_df.head()

,close
2016-06-30,78.846061
2016-09-30,82.793553
2016-12-31,75.987303
2017-03-31,78.045968
2017-06-30,79.547540


### CASE-SCHILLER HOUSE INDEX DATA via FRED

In [6]:
!pip install fredapi

In [7]:
# initial imports
from fredapi import Fred

In [8]:
load_dotenv()

True

In [9]:
fred_api = "0316b214fd1344467d844a5fc2236bf0"
#os.getenv('FRED_API_KEY')
fred = Fred(api_key=fred_api)

In [10]:
print (fred_api)

0316b214fd1344467d844a5fc2236bf0


In [22]:
#df_CS = {}
df_CS = fred.get_series('CSUSHPINSA')

In [27]:
df_CS = pd.DataFrame(df_CS, index=df_CS.index)
df_CS.index = pd.to_datetime(df_CS.index)
df_CS.columns =['Case-Shiller U.S. National Home Price Index']
df_CS = df_CS.dropna()
df_CS = df_CS.resample('Q').mean()

df_CS.head()

,Case-Shiller U.S. National Home Price Index
1987-03-31,64.115333
1987-06-30,65.583667
1987-09-30,67.226667
1987-12-31,68.114333
1988-03-31,68.941000


### The Features

In [13]:
# Initial Imports